In [1]:
import os
from os import path
from datetime import datetime, timedelta, date
import pytz

from igraph import *

import seaborn as sns
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.colors as clr
%matplotlib inline

In [2]:
title = "WKU"
base_folder = "./simulations/wku23"
sim_id = 165
sim_tz = "Asia/Shanghai"
time0 = 'Nov 20 2023 9:00AM'
time1 = 'Dec 4 2023 12:00PM'
time_step_min = 30
use_new_id_schema = True   